Cada vez que hagamos una consulta/agregación, se indicará el esquema de la respuesta. Para devolver el mismo nombre, simplemente al hacer la proyección (tanto en consultas como agregaciones), incluís el nombre del campo nuevo como clave, y el antiguo como valor utilizando `$`.

Para importar una colección desde un archivo `JSON`, utiliza el siguiente comando:

```sh
mongoimport --db nombre_base_de_datos --collection nombre_coleccion --file archivo_json
```

# Ejercicio 1: Gestor FdIx (CRUD/Agregaciones)

Vamos a montar nuestro propio servicio de _streaming_ en la Facultad de Informática, _FdIx_. Nuestra base de datos en MongoDB cuenta con tres colecciones: _movies_, la cual contiene la lista disponible de películas; _users_, que contiene la lista de usuarios registrados; y _comments_, la cual contiene una lista de comentarios de nuestras películas.

Datos Generales:

* Nombre Base de Datos: `fdix`

* Nombre Archivo 1: `movies.json`
* Nombre Colección 1: `movies`

* Nombre Archivo 2: `users.json`
* Nombre Colección 2: `users`

* Nombre Archivo 3: `comments.json`
* Nombre Colección 3: `comments`

El primer paso consiste en familiarizarse con el esquema de datos que siguen estas colecciones. Para ello, prueba a hacer un `findOne` de cada una de las colecciones y haz un pequeño esquema de cada colección. A continuación, crea una clase `GestionFdIx` en la que vamos a implementar las siguientes operaciones:

## 1.1 Inserciones

Implementa una función independiente de la clase `GestionFdIx` que devuelva un comentario en formato `JSON`, utilizando como fecha la fecha de hoy (`datetime.datetime.today()`):

```python
def generar_comentario(name: str, email: str,
                       movie_id: str, text: str) -> Dict[str, Any]:
```

A continuación, crea una función que devuelva `n` comentarios aleatorios invocando al método anterior. Para ello, selecciona un usuario y una película aleatoriamente, e introduce un comentario con texto aleatorio. 

```python
def comentarios_aleatorios(self, n: int) -> List[Dict[str, Any]]:
```

Para insertar un texto aleatorio, se puede utilizar la siguiente función de la librería `lorem-text`:

```python
from lorem_text import lorem

lorem.paragraph()
```

Por último, crea una función para insertar una serie de comentarios aleatorios:

```python
def insertar_comentarios_aleatorios(self, n) -> Dict[str, Any]:
```

Además, se debe actualizar el campo `num_fdix_comments` de la colección `movies` de las películas correspondientes.

## 1.2 Borrados

Crea una función para borrar todos los comentarios de un usuario dado su email en un rango de fechas, siguiendo el siguiente formato:

```python
def borrar_comentarios(self, email: str, fecha_inicial: datetime.datetime,
                       fecha_final: datetime.datetime) -> Dict[str, Any]:
```

Como parte de la consulta, debemos también actualizar el campo `num_fdix_comments` de la colección `movies` de las películas correspondientes.

## 1.3 Actualizaciones

### 1.3.1 Actualizar Nombre

Crea una función que cambie el nombre asociado a un email. Para ello, hay que actualizar tanto la colección de usuarios como la de comentarios.

```python
def actualizar_nombre(self, email: str, nombre_nuevo: str) -> Tuple[Dict[str, Any], Dict[str, Any]]:
```

Prueba a actualizar el nombre del email `sean_bean@gameofthron.es` por `Ned Starkn't`.

### 1.3.2 Actualizar Lenguajes

Crea una función que, dado el título de una película, añada un nuevo lenguaje a la lista de lenguajes si no lo tenía ya disponible previamente.

```python
def nuevo_lenguaje(self, titulo: str, idioma: str):
```

Por ejemplo, prueba a añadir `Spanish` a la película `The Perils of Pauline`.

### 1.3.3 Reemplazar un Documento

Dado el id de un comentario, reemplaza el comentario especificado por uno generado aleatoriamente (utilizando la función `generar_comentario`). Comprueba que efectivamente se ha llevado a cabo el reemplazo.

```python
def reemplazar_comentario(self, id_comentario: str):
```

Por ejemplo, prueba con `id_comentario` "573a1390f29313caabcd5b9a".

## 1.4 Consultas

Implementa las siguientes funciones para consultar información sobre películas:

### Consulta 1

Devuelve las `n` últimas películas que estén disponibles en una lista de idiomas dado, ordenadas por fecha de estreno. 

```python
def encontrar_peliculas_idioma(self, n: int, lista_idiomas: List[str]) -> List[Dict[str, Any]]:
```
Por ejemplo, probad a devolver las últimas 10 películas que están disponibles tanto en inglés (`English`) como en francés (`French`).

Esquema: (**titulo**, **idiomas**, **fecha**).

### Consulta 2

Encuentra las películas en las posiciones `i` y `j` (inclusive), ordenadas de forma descendente según las puntuaciones de `imdb`.

```python
def mejor_valoradas(self, i: int, j: int) -> List[Dict[str, Any]]:
```

Nótese que en la colección hay películas que no tienen valoraciones y que no hay que considerar, representadas como `''` en el campo correspondiente. Prueba a devolver las películas mejor valoradas de la posición 10 a la 20.

Esquema: (**titulo**, **rating**).

### Consulta 3

Encuentra las películas que tienen un rating de más de 4 estrellas y con más de 1000 reseñas en **rotten-tomatoes** según los usuarios, ordenadas por el rating, seguido del número de reseñas de forma descendente.

```python
def peliculas_populares(self) -> List[Dict[str, Any]]:
```

Esquema: (**titulo**, **estrellas**, **numeroValoraciones**).

### Consulta 4

Encuentras las películas que hayan ganado en todas las categorías en las que ha estado nominada, ordenada por número de premios de forma descendente y por `_id`:

```python
def ganar_categoria(self) -> List[Dict[str, Any]]:
```

Esquema: (**titulo**, **numeroPremios**)

### Consulta 5 (DIFÍCIL)

Encuentra las películas que se han estrenado en un año dado en las que la diferencia entre el rating del público y de los críticos sea mayor, ordenadas por esta diferencia. Debéis comprobar que la película correspondiente tiene un rating asociado a críticos y a usuarios. Para ello, utilizad la operación `{"exists": True}`. También deberéis utilizar las operaciones `$abs` y `$substract`:

```python
def mayor_diferencia_ratings(self, anyo: int) -> List[Dict[str, Any]]:
```

Esquema: (**titulo**, **ratingPublico**, **ratingCriticos**, **diferencia**)

## 1.5 Índices

Dadas las consultas del apartado `1.4`, ¿qué índices consideras que se deberían crear para mejorar el rendimiento? Crea los índices correspondientes en la colección `movies`.

## 1.6 Agregaciones

#### Agregación 1

Identifica cuántos elementos tiene cada categoría de producción audiovisual (campo `type`), ordenadas por el número de elementos de manera ascendente. En lugar de devolver la información en una lista de diccionarios, combina toda la información en un único diccionario, donde la clave corresponde a la categoría de producción; y el valor, al número total de películas en esa categoría.

```python
def numero_peliculas_por_categoria(self) -> Dict[str, int]
```

Esquema: {**categoria**: **numeroPeliculas**}

#### Agregación 2

Devuelve el número de elementos que hay asociados a un género de película dado.

```python
def numero_peliculas_genero(self, genero: str) -> int
```

Esquema: **numeroPeliculas**

#### Agregación 3

Devuelve la lista de usuarios (`email` y `nombre`) que han comentado en una película dada:

```python
def han_comentado(self, titulo: str) -> List[Dict[str, str]]:
```

Esquema: (**email**, **nombre**)

# Ejercicio 2: Ventas (Consultas/Agregaciones)

En este ejercicio, se puede utilizar tanto la terminal de Mongo, `mongosh`, como realizar las agregaciones desde Python directamente con `PyMongo`. Os recomiendo que os familiaricéis también con `mongosh`.

Datos Generales:

* Nombre Archivo: `ventas.json`
* Nombre Base de Datos: `ventas`
* Nombre Colección: `ventas`

Desde nuestra querida empresa, **Universal Corporate Materials**, nos han pedido que recortemos el coste en material de cara a nuestros siguientes pedidos (_El coste de la vida está muy difícil_, nos han argumentado). Para ello, nos han proporcionado la información de los últimos 5000 pedidos que han realizado, y nos han pedido que averigüemos la siguiente información:

### Tarea 1: Estudiar la Distribución de Datos

En primer lugar, queremos entender más a fondo cómo se han distribuido las distintas compras que se han realizado. Para ello, realizaremos distintas consultas sobre los datos de ventas:

### Pedidos más recientes

Devuelve la localización de la tienda, el email del comprador y el método de compra de los últimos 10 pedidos. 

Esquema: (**localizacionTienda**, **fecha**, **email**, **metodoCompra**).

### Producto más Comprado

Devuelve uno de los productos que más se hayan comprado. 

Esquema: (**nombreProducto**, **cantidad**)

### Producto más Caro en Categoría

Devuelve el producto más caro de la categoría `office` entre todos los pedidos que se han realizado. Tened en cuenta que queremos hallar el producto más caro por **unidad**. 

Esquema: (**nombreProducto**, **idPedido**, **precioProducto**)

**NOTA**: Para hallar el precio por producto, podéis utilizar el operador `$divide` en la fase de `$project`, el cual recibe una lista con el numerador y el denominador.

### Información por Tipo de Producto

Muestra el precio total de los productos que se han comprado por cada categoría de `tags`, y el número de productos total en cada categoría. Esquema: (**categoria**, **total_precio**, **cantidad**).

### Localización Mejor Valorada

Devuelve la localización que tiene la mejor valoración, teniendo en cuenta las siguientes consideraciones:

* Para determinar la mejor valoración, haremos la media de las valoraciones medias por cada usuario.
* Solo consideraremos tiendas que han sido al menos valoradas por 10 usuarios distintos.


**NOTA**: acordaros que podéis agrupar el `_id` de la operación `$group` en función de un objeto si necesitáis agrupar varias claves.

Esquema: (**localizacionTienda**, **valoracion**, **numeroValoraciones**)

## Tarea 2: Filtrar Actividad Sospechosa para Activar el Bonus de Ahorro (DIFÍCIL)

En **Universal Corporate Materials**, los empleados reciben un bonus cada 5000 pedidos en función de la cantidad de dinero que han conseguido ahorra utilizando cupones. Se sospecha que hay varios empleados que han cometido prácticas sospechosas para activar el umbral e intentar embolsarse la mayor cantidad de dinero posible. Vamos a estudiar los siguientes casos:

### Pedidos Muy Poco Costosos

Desde nuestra empresa, se recomienda hacer pedidos cuya cantidad sume al menos 30 dólares, ya que de lo contrario incurrimos en gastos de transporte muy altos. Hay empleados que están haciendo pedidos de muy poco importe para llegar a los 5000 pedidos. Para detectar qué empleados están manipulando artificialmente el número de pedidos, vamos a estudiar la distribución de los mismos. 

Escribe una agregación que devuelva el número de pedidos que sumen menos de 30 dólares por cada empleado (identificado por su `email`), ordenados de forma descendente por este número, y en caso de empate, de forma ascendente por el email.

Esquema: (**email**, **numeroPedidos**)

### Pedidos Repetidos al Mismo Proveedor

Otra actividad que queremos frenar son los pactos encubiertos entre proveedores y empleados, de tal manera que el empleado se embolsa un porcentaje de la compra. Todos los proveedores venden el mismo tipo de productos, así que lo más natural es que un empleado realiza compras a diversos proveedores, aprovechando ofertas. 

Escribe una agregación en la que se devuelva por cada empleado (identificado por su email) cuál es el porcentaje de compras del mayor proveedor. Aquí vas a necesitar el operador `$divide`, el cual se aplica en la fase de `$project` y recibe como parámetro una lista con dos elementos: el numerador y el denominador.

Esquema: (**email**, **porcentaje**)

### Pedidos con Productos Muy Caros (MUY DIFÍCIL)

Siguiendo la misma idea de antes, también queremos detectar aquellas compras en las que el precio del producto comprado sea muy superior al de la media de ese mismo producto en todos los productos. La compra de un producto es sospechosa si es al menos un 50% superior que la media.

Escribe una agregación que detecte qué compras de productos son sospechosas y quién ha realizado esas compras. Para ello, realizad dos agregaciones: una para computar el precio medio por cada producto, la cual almacenáis en una colección intermedia `media_productos`. Utilizando esta colección intermedia, comprobad quién ha realizado compras sospechosas y cuántas ha realizado.

**NOTA**: puede que necesites usar el operador `$expr` para introducir una comparación entre distintos campos del documento, seguido del operador que se quiere aplicar. Por ejemplo, `$expr: { $gt: [ "$campo1" , "$campo2"]`.

Esquema: (**email**, **numeroCompras**)

## Ahorro Utilizando Cupones

Una vez hemos detectado la actividad sospechosa, vamos a calcular el bonus que se aplica a los usuarios que no han incurrido en actividades fraudulentas. La política de empresa es reembolsar un 1% del total del precio de compras con cupón al usuario que ha realizado esa compra. Realizad una agregación que devuelva esta información.

Para ello, vamos a crear una función en Python que dada una lista de usuarios sospechosos y la colección de ventas, devuelva el bonus aplicado al resto. El esquema es:

```python
def reembolso(coleccion, lista_sospechosos: List[str]) -> Dict[str, float]
```

la cual devuelve un diccionario con la cantidad a reembolsar a cada empleado que no forma parte de `lista_sospechosos`.
